In [ ]:
# from google.colab import drive

In [ ]:
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
path = '../data/processed/engineered_features.csv'
df = pd.read_csv(path)
df.head()

,customerID,customerType,riskLevel,investmentCapacity,account_creation_date,ISIN,transactionID,transactionType,transaction_date,totalValue,...,days_since_last_trade,holding_duration_days,cust_asset_total_value,cust_asset_n_txn,cust_asset_avg_value,cust_asset_buy_ratio,last_cust_asset_txn,cust_asset_days_since_last_txn,cust_total_value,cust_asset_portfolio_share
0,00017496858921195E5A,Professional,Aggressive,CAP_GT300K,2021-03-19,GRS434003000,7590224,Buy,2020-03-27,11000.0,...,0,1793,229180.0,12,19098.333333,0.5,2021-05-19,560,728451.013,0.314613
1,00017496858921195E5A,Professional,Aggressive,CAP_GT300K,2021-03-19,GRS434003000,7607029,Sell,2020-04-06,12080.0,...,0,1793,229180.0,12,19098.333333,0.5,2021-05-19,560,728451.013,0.314613
2,00017496858921195E5A,Professional,Aggressive,CAP_GT300K,2021-03-19,GRS434003000,7634872,Buy,2020-04-24,13400.0,...,0,1793,229180.0,12,19098.333333,0.5,2021-05-19,560,728451.013,0.314613
3,00017496858921195E5A,Professional,Aggressive,CAP_GT300K,2021-03-19,GRS434003000,7652627,Sell,2020-05-07,12700.0,...,0,1793,229180.0,12,19098.333333,0.5,2021-05-19,560,728451.013,0.314613
4,00017496858921195E5A,Professional,Aggressive,CAP_GT300K,2021-03-19,GRS434003000,7664807,Buy,2020-05-15,12150.0,...,0,1793,229180.0,12,19098.333333,0.5,2021-05-19,560,728451.013,0.314613


In [ ]:
df.columns

Index(['customerID', 'customerType', 'riskLevel', 'investmentCapacity',
       'account_creation_date', 'ISIN', 'transactionID', 'transactionType',
       'transaction_date', 'totalValue', 'units', 'channel', 'marketID',
       'assetName', 'assetCategory', 'assetSubCategory', 'sector', 'industry',
       'asset_description', 'exchangeID', 'name', 'market_description',
       'country', 'tradingHours', 'marketClass', 'days_since_txn',
       'value_per_unit', 'is_buy', 'is_sell', 'channel_encoded',
       'account_age_days', 'risk_encoded', 'capacity_numeric',
       'n_unique_assets', 'n_unique_sectors', 'n_unique_markets',
       'total_invested', 'avg_txn_value', 'median_units', 'n_transactions_x',
       'n_buys_x', 'n_sells_x', 'buy_ratio_x', 'channel_pref', 'last_txn_date',
       'days_since_last_txn', 'risk_level', 'invest_capacity',
       'portfolio_concentration', 'day_of_week', 'month', 'quarter',
       'n_unique_investors', 'total_traded_value', 'avg_traded_value',
      

In [ ]:
df_pos = df[df["is_buy"]==1].copy()
df_pos["user_id"] = df_pos["customerID"]
df_pos["item_id"] = df_pos["ISIN"]
ts = pd.to_datetime(df_pos["transaction_date"], errors="coerce")
df_pos = df_pos[ts.notna()].copy()
df_pos["timestamp"] = ts[ts.notna()].astype("int64") // 10**9
df_pos["rating"] = 1
df_pos = df_pos[["user_id", "item_id", "timestamp", "rating"]].sort_values("timestamp")
df_pos = df_pos.drop_duplicates(["user_id", "item_id"], keep="last")


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
DEVICE

device(type='cuda')

In [ ]:
EMBED_GMF = 32
EMBED_MLP = 32
MLP_LAYERS = (128, 64, 32)
NEG_PER_POS = 4
BATCH_SIZE = 2048
EPOCHS = 10
LR = 1e-3
WEIGHT_DECAY = 1e-6
TOPK = 10

In [ ]:
df_cf = df_pos[df_pos["rating"]>0].copy()
test_df = df_cf.groupby("user_id").tail(1)
train_df = df_cf.drop(test_df.index)

In [ ]:
user_all = pd.concat([train_df["user_id"], test_df["user_id"]])
item_all = pd.concat([train_df["item_id"], test_df["item_id"]])
user2id = {u: i for i, u in enumerate(user_all.astype("category").cat.categories)}
item2id = {i: j for j, i in enumerate(item_all.astype("category").cat.categories)}
train_df["u"] = train_df["user_id"].map(user2id).astype(int)
train_df["i"] = train_df["item_id"].map(item2id).astype(int)
test_df["u"] = test_df["user_id"].map(user2id).astype(int)
test_df["i"] = test_df["item_id"].map(item2id).astype(int)

/tmp/ipython-input-3811300264.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["u"] = test_df["user_id"].map(user2id).astype(int)
/tmp/ipython-input-3811300264.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["i"] = test_df["item_id"].map(item2id).astype(int)


In [ ]:
n_users = len(user2id)
n_items = len(item2id)
print(n_users, n_items)

29090 320


In [ ]:
user_pos = defaultdict(set)
for u,i in zip(train_df.u.values, train_df.i.values):
    user_pos[int(u)].add(int(i))

In [ ]:
class PairwiseImplicitDataset(Dataset):
    def __init__(self, df, n_items, user_pos, neg_per_pos=4):
        import numpy as np
        self.user_features = ["risk_encoded", "capacity_numeric", "channel_encoded", "account_age_days"]
        self.item_features = ["assetCategory", "sector", "industry", "marketClass"]

        missing_user = [c for c in self.user_features if c not in df.columns]
        missing_item = [c for c in self.item_features if c not in df.columns]
        for c in ["u", "i"]:
            if c not in df.columns:
                missing_user.append(c)
        if missing_user or missing_item:
            raise ValueError(f"Missing columns. user: {missing_user}, item: {missing_item}")

        df = df.reset_index(drop=True)

        self.users = df["u"].to_numpy(dtype=np.int64)
        self.items = df["i"].to_numpy(dtype=np.int64)
        self.uf_mat = df[self.user_features].to_numpy(dtype=np.float32)
        self.if_mat = df[self.item_features].to_numpy(dtype=np.float32)

        self.n_items = int(n_items)
        self.user_pos = user_pos
        self.neg_per_pos = int(neg_per_pos)

    def __len__(self):
        return self.users.shape[0]

    def __getitem__(self, idx):
        import numpy as np
        u = int(self.users[idx])
        p = int(self.items[idx])
        uf = self.uf_mat[idx]
        ifeat = self.if_mat[idx]

        # negative sampling
        liked = self.user_pos.get(u, set())
        negs = []
        if len(liked) < self.n_items - 1:
            comp = np.setdiff1d(
                np.arange(self.n_items, dtype=np.int64),
                np.fromiter(liked, dtype=np.int64, count=len(liked)),
                assume_unique=False
            )
            comp = comp[comp != p]
            if comp.size >= self.neg_per_pos:
                negs = np.random.choice(comp, size=self.neg_per_pos, replace=False).astype(np.int64)
            else:
                negs = np.random.choice(comp, size=self.neg_per_pos, replace=True).astype(np.int64)
        else:
            while len(negs) < self.neg_per_pos:
                j = np.random.randint(0, self.n_items, dtype=np.int64)
                if j != p and j not in liked:
                    negs.append(j)
            negs = np.asarray(negs, dtype=np.int64)

        return (
            torch.as_tensor(u, dtype=torch.long),
            torch.as_tensor(p, dtype=torch.long),
            torch.as_tensor(negs, dtype=torch.long),
            torch.from_numpy(uf),
            torch.from_numpy(ifeat),
        )


In [ ]:
print(train_df.columns.tolist())


['user_id', 'item_id', 'timestamp', 'rating', 'u', 'i']


In [ ]:
# Feature Prep + Safe Merges (drop-in replacement)
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import numpy as np

# 1) Declare feature columns
user_features = ["risk_encoded", "capacity_numeric", "channel_encoded", "account_age_days"]
item_features = ["assetCategory", "sector", "industry", "marketClass"]

# 2) Work on a copy of the master table to avoid side effects
df_enc = df.copy()

# 3) Encode item categorical columns if needed (no-op for numeric)
for col in item_features:
    if df_enc[col].dtype == "object":
        le = LabelEncoder()
        df_enc[col] = le.fit_transform(df_enc[col].astype(str))

# 4) Build per-user table and scale user features WITHOUT leakage
user_base = df_enc.drop_duplicates("customerID")[["customerID"] + user_features].copy()
scaler = MinMaxScaler()

# Fit on users present in train_df, then transform all users
train_users = train_df["user_id"].unique()
mask_train = user_base["customerID"].isin(train_users)

user_base.loc[mask_train, user_features] = scaler.fit_transform(
    user_base.loc[mask_train, user_features]
)
if (~mask_train).any():
    user_base.loc[~mask_train, user_features] = scaler.transform(
        user_base.loc[~mask_train, user_features]
    )

# 5) Build per-item table (already numerically encoded above)
item_base = df_enc.drop_duplicates("ISIN")[["ISIN"] + item_features].copy()

# 6) Clean old columns (prevents _x/_y suffix conflicts on reruns)
train_df = train_df.drop(columns=user_features + item_features, errors="ignore").copy()
test_df  = test_df.drop(columns=user_features + item_features, errors="ignore").copy()

# 7) Merge user + item features into train/test
train_df = train_df.merge(user_base, left_on="user_id", right_on="customerID", how="left")
train_df = train_df.merge(item_base, left_on="item_id", right_on="ISIN", how="left")
train_df.drop(columns=["customerID", "ISIN"], inplace=True, errors="ignore")
train_df[user_features + item_features] = (
    train_df[user_features + item_features].fillna(0).astype("float32")
)

test_df = test_df.merge(user_base, left_on="user_id", right_on="customerID", how="left")
test_df = test_df.merge(item_base, left_on="item_id", right_on="ISIN", how="left")
test_df.drop(columns=["customerID", "ISIN"], inplace=True, errors="ignore")
test_df[user_features + item_features] = (
    test_df[user_features + item_features].fillna(0).astype("float32")
)

# 8) Sanity checks
assert set(user_features) <= set(train_df.columns), "User features missing in train_df"
assert set(item_features) <= set(train_df.columns), "Item features missing in train_df"
assert set(user_features) <= set(test_df.columns),  "User features missing in test_df"
assert set(item_features) <= set(test_df.columns),  "Item features missing in test_df"

print("Feature merge complete.")


/tmp/ipython-input-2650109063.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.5 1.  0.5 ... 0.  0.  0. ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  user_base.loc[mask_train, user_features] = scaler.fit_transform(
/tmp/ipython-input-2650109063.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.16908898 0.2112726  0.2112726  ... 0.2112726  0.2112726  0.28571429]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  user_base.loc[mask_train, user_features] = scaler.fit_transform(


Feature merge complete.


In [ ]:
# make sure these exist: train_df, n_items, user_pos, NEG_PER_POS, BATCH_SIZE
train_df = train_df.reset_index(drop=True)

train_loader = DataLoader(
    PairwiseImplicitDataset(train_df, n_items, user_pos, NEG_PER_POS),
    batch_size=BATCH_SIZE,
    shuffle=True,
    drop_last=False
)


In [ ]:
class NeuMF(nn.Module):
    def __init__(self, n_users, n_items, eg=32, em=32, layers=(128,64,32)):
        super().__init__()
        # GMF branch
        self.user_gmf = nn.Embedding(n_users, eg)
        self.item_gmf = nn.Embedding(n_items, eg)
        # MLP branch
        self.user_mlp = nn.Embedding(n_users, em)
        self.item_mlp = nn.Embedding(n_items, em)
        dim = em*2
        mlp = []
        for h in layers:
            mlp += [nn.Linear(dim, h), nn.ReLU()]
            dim = h
        self.mlp = nn.Sequential(*mlp)
        # fusion
        self.out = nn.Linear(eg + layers[-1], 1)
        self.sig = nn.Sigmoid()
        self._init()
    def _init(self):
        for emb in [self.user_gmf, self.item_gmf, self.user_mlp, self.item_mlp]:
            nn.init.normal_(emb.weight, std=0.01)
        for m in self.mlp:
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight); nn.init.zeros_(m.bias)
        nn.init.xavier_uniform_(self.out.weight); nn.init.zeros_(self.out.bias)
    def forward(self, u, i, uf=None, ifeat=None):
      g = self.user_gmf(u) * self.item_gmf(i)
      m = self.mlp(torch.cat([self.user_mlp(u), self.item_mlp(i)], dim=-1))

      if uf is not None and ifeat is not None:
          x = torch.cat([g, m, uf, ifeat], dim=-1)
          out = self.out_ext(x)
      else:
          out = self.out(torch.cat([g, m], dim=-1))

      return self.sig(out.squeeze(-1))


In [ ]:
def bce_pos_neg(model, users, pos_items, neg_items):
    B, K = neg_items.shape
    users = users.to(DEVICE); pos_items = pos_items.to(DEVICE); neg_items = neg_items.to(DEVICE)
    pos_scores = model(users, pos_items)
    users_rep = users.unsqueeze(1).expand(-1, K).reshape(-1)
    neg_scores = model(users_rep, neg_items.reshape(-1)).view(B, K)
    loss = nn.functional.binary_cross_entropy(pos_scores, torch.ones_like(pos_scores))
    loss += nn.functional.binary_cross_entropy(neg_scores, torch.zeros_like(neg_scores))
    return loss

In [ ]:
@torch.no_grad()
def evaluate(model, test_df, user_pos, n_items, k=10, n_neg=99):
    model.eval()
    HR, NDCG = [], []
    for u, pos in zip(test_df.u.values, test_df.i.values):
        u = int(u); pos = int(pos)
        negs = []
        s = user_pos[u]
        while len(negs) < n_neg:
            j = np.random.randint(0, n_items)
            if j != pos and j not in s:
                negs.append(j)
        items = np.array([pos] + negs, dtype=np.int64)
        users = np.full_like(items, u)
        scores = model(
            torch.tensor(users, device=DEVICE),
            torch.tensor(items, device=DEVICE)
        ).cpu().numpy()
        rank = scores.argsort()[::-1]
        topk = items[rank[:k]]
        HR.append(1 if pos in topk else 0)
        if pos in topk:
            idx = np.where(topk == pos)[0][0]
            NDCG.append(1 / np.log2(idx + 2))
        else:
            NDCG.append(0.0)
    return float(np.mean(HR)), float(np.mean(NDCG))


In [ ]:

model = NeuMF(n_users, n_items, EMBED_GMF, EMBED_MLP, MLP_LAYERS).to(DEVICE)
opt = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

for e in range(1, EPOCHS + 1):
    model.train()
    total = 0.0

    for u, p, n, _, _ in train_loader:
        u = u.long().to(DEVICE)
        p = p.long().to(DEVICE)
        n = n.long().to(DEVICE)

        opt.zero_grad()
        loss = bce_pos_neg(model, u, p, n)
        loss.backward()
        opt.step()
        total += loss.item() * len(u)

    # Evaluate after each epoch (features not used by model)
    hr, ndcg = evaluate(model, test_df, user_pos, n_items, k=TOPK)

    print(f"Epoch {e:02d} | train_bce={total/len(train_df):.4f} | "
          f"HR@{TOPK}={hr:.3f} | NDCG@{TOPK}={ndcg:.3f}")


Epoch 01 | train_bce=1.2890 | HR@10=0.601 | NDCG@10=0.294
Epoch 02 | train_bce=0.8442 | HR@10=0.636 | NDCG@10=0.393
Epoch 03 | train_bce=0.7430 | HR@10=0.632 | NDCG@10=0.399
Epoch 04 | train_bce=0.7044 | HR@10=0.641 | NDCG@10=0.405
Epoch 05 | train_bce=0.6746 | HR@10=0.672 | NDCG@10=0.417
Epoch 06 | train_bce=0.6526 | HR@10=0.694 | NDCG@10=0.426
Epoch 07 | train_bce=0.6243 | HR@10=0.693 | NDCG@10=0.426
Epoch 08 | train_bce=0.6004 | HR@10=0.693 | NDCG@10=0.429
Epoch 09 | train_bce=0.5705 | HR@10=0.705 | NDCG@10=0.433
Epoch 10 | train_bce=0.5387 | HR@10=0.729 | NDCG@10=0.455


In [ ]:

user_features = ["risk_encoded", "capacity_numeric", "channel_encoded", "account_age_days"]
item_features = ["assetCategory", "sector", "industry", "marketClass"]

from sklearn.preprocessing import LabelEncoder, MinMaxScaler

df_enc = df.copy()  # avoid mutating original df


labelers = {}
for col in item_features:
    if df_enc[col].dtype == "object":
        le = LabelEncoder()
        df_enc[col] = le.fit_transform(df_enc[col].astype(str))
        labelers[col] = le


scaler = MinMaxScaler()

# Build per-user table first from FULL df_enc (raw), then fit on train users only
user_base = df_enc.drop_duplicates("customerID")[["customerID"] + user_features].copy()

train_users = train_df["user_id"].unique()
mask_train_users = user_base["customerID"].isin(train_users)

# Fit on train users only, then transform all users for consistency
user_base.loc[mask_train_users, user_features] = scaler.fit_transform(
    user_base.loc[mask_train_users, user_features]
)
user_base.loc[~mask_train_users, user_features] = scaler.transform(
    user_base.loc[~mask_train_users, user_features]
)


item_base = df_enc.drop_duplicates("ISIN")[["ISIN"] + item_features].copy()


train_df = train_df.drop(columns=user_features + item_features, errors="ignore").copy()
test_df  = test_df.drop(columns=user_features + item_features, errors="ignore").copy()

train_df = train_df.merge(user_base, left_on="user_id", right_on="customerID", how="left")
train_df = train_df.merge(item_base, left_on="item_id", right_on="ISIN", how="left")
train_df.drop(columns=["customerID", "ISIN"], inplace=True, errors="ignore")

test_df = test_df.merge(user_base, left_on="user_id", right_on="customerID", how="left")
test_df = test_df.merge(item_base, left_on="item_id", right_on="ISIN", how="left")
test_df.drop(columns=["customerID", "ISIN"], inplace=True, errors="ignore")


train_df[user_features + item_features] = train_df[user_features + item_features].fillna(0).astype("float32")
test_df[user_features + item_features]  = test_df[user_features + item_features].fillna(0).astype("float32")

# Sanity checks
assert set(user_features) <= set(train_df.columns)
assert set(item_features) <= set(train_df.columns)


/tmp/ipython-input-2856375123.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.5 1.  0.5 ... 0.  0.  0. ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  user_base.loc[mask_train_users, user_features] = scaler.fit_transform(
/tmp/ipython-input-2856375123.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.16908898 0.2112726  0.2112726  ... 0.2112726  0.2112726  0.28571429]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  user_base.loc[mask_train_users, user_features] = scaler.fit_transform(


In [ ]:
from torch.utils.data import Dataset
import numpy as np
import torch

class ContextualImplicitDataset(Dataset):
    def __init__(self, df, n_items, user_pos, neg_per_pos=4, max_tries=1000):
        # Ensure row order is consistent
        df = df.reset_index(drop=True)

        # Integer ids for embeddings
        self.users = df["u"].to_numpy(dtype=np.int64)
        self.items = df["i"].to_numpy(dtype=np.int64)

        # Context features (float32)
        self.user_feats = df[user_features].to_numpy(dtype=np.float32)
        self.item_feats = df[item_features].to_numpy(dtype=np.float32)

        self.n_items = int(n_items)
        self.user_pos = user_pos
        self.neg_per_pos = int(neg_per_pos)
        self.max_tries = int(max_tries)

    def __len__(self):
        return len(self.users)

    def _sample_negatives(self, u, pos):
        liked = self.user_pos.get(u, set())
        if pos in liked:
            pass

        # Universe minus liked
        if len(liked) < self.n_items - 1:
            comp = np.setdiff1d(np.arange(self.n_items, dtype=np.int64),
                                np.fromiter(liked, dtype=np.int64, count=len(liked)),
                                assume_unique=False)
            # Also exclude current positive just to be safe
            comp = comp[comp != pos]
            if comp.size >= self.neg_per_pos:
                return np.random.choice(comp, size=self.neg_per_pos, replace=False).astype(np.int64)
            # Fallback to allowing replacement if universe is tiny
            return np.random.choice(comp, size=self.neg_per_pos, replace=True).astype(np.int64)

        negs = []
        tries = 0
        while len(negs) < self.neg_per_pos and tries < self.max_tries:
            j = np.random.randint(0, self.n_items, dtype=np.int64)
            if j != pos and j not in liked:
                negs.append(j)
            tries += 1
        # If still short, fill randomly from all except pos (may include collisions with liked if unavoidable)
        if len(negs) < self.neg_per_pos:
            pool = np.setdiff1d(np.arange(self.n_items, dtype=np.int64), np.array([pos], dtype=np.int64))
            fill = np.random.choice(pool, size=self.neg_per_pos - len(negs), replace=True).astype(np.int64)
            negs.extend(fill.tolist())
        return np.array(negs, dtype=np.int64)

    def __getitem__(self, idx):
        u = self.users[idx]
        pos = self.items[idx]
        uf = self.user_feats[idx]
        ifeat = self.item_feats[idx]

        negs = self._sample_negatives(u, pos)

        # Return tensors with desired dtypes; DataLoader will batch them
        return (
            torch.as_tensor(u, dtype=torch.long),
            torch.as_tensor(pos, dtype=torch.long),
            torch.as_tensor(negs, dtype=torch.long),
            torch.from_numpy(uf),
            torch.from_numpy(ifeat),
        )


In [ ]:
class AdvancedNeuMF(nn.Module):
    def __init__(self, n_users, n_items, user_feat_dim, item_feat_dim,
                 eg=32, em=32, layers=(128, 64, 32), use_bn=True):
        super().__init__()
        # Embeddings
        self.user_gmf = nn.Embedding(n_users, eg)
        self.item_gmf = nn.Embedding(n_items, eg)
        self.user_mlp = nn.Embedding(n_users, em)
        self.item_mlp = nn.Embedding(n_items, em)

        # Project raw context
        self.user_fc = nn.Linear(user_feat_dim, 16)
        self.item_fc = nn.Linear(item_feat_dim, 16)

        # MLP over [user_mlp, item_mlp, projected_user_ctx, projected_item_ctx]
        dim = em*2 + 32
        mlp = []
        for h in layers:
            mlp.append(nn.Linear(dim, h))
            if use_bn: mlp.append(nn.BatchNorm1d(h))
            mlp += [nn.ReLU(), nn.Dropout(0.2)]
            dim = h
        self.mlp = nn.Sequential(*mlp)

        # Fusion of GMF and MLP tower
        self.out = nn.Linear(eg + layers[-1], 1)

        self._init()

    def _init(self):
        for emb in [self.user_gmf, self.item_gmf, self.user_mlp, self.item_mlp]:
            nn.init.normal_(emb.weight, std=0.01)
        nn.init.xavier_uniform_(self.user_fc.weight); nn.init.zeros_(self.user_fc.bias)
        nn.init.xavier_uniform_(self.item_fc.weight); nn.init.zeros_(self.item_fc.bias)
        nn.init.xavier_uniform_(self.out.weight);     nn.init.zeros_(self.out.bias)

    def forward(self, u, i, uf, ifeat):

        g = self.user_gmf(u) * self.item_gmf(i)  # (B, eg)

        # project context, then concatenate into MLP stream
        u_ctx = self.user_fc(uf)
        i_ctx = self.item_fc(ifeat)

        m_in = torch.cat([self.user_mlp(u), self.item_mlp(i), u_ctx, i_ctx], dim=-1)
        m = self.mlp(m_in)

        logits = self.out(torch.cat([g, m], dim=-1)).squeeze(-1)
        return logits


In [ ]:
import torch.nn.functional as F

def context_bce(model, users, pos_items, neg_items, u_feats, pos_i_feats, item_feat_matrix):

    B, K = neg_items.shape


    pos_logits = model(users, pos_items, u_feats, pos_i_feats)   # (B,)
    pos_loss = F.binary_cross_entropy_with_logits(
        pos_logits, torch.ones_like(pos_logits)
    )


    # replicate users & user features for K negatives
    users_rep  = users.unsqueeze(1).expand(-1, K).reshape(-1)
    u_feats_rep = u_feats.unsqueeze(1).expand(-1, K, -1).reshape(-1, u_feats.shape[1])

    # flatten neg ids and lookup their own item features
    n_flat = neg_items.reshape(-1)
    neg_i_feats = item_feat_matrix[n_flat]

    neg_logits = model(users_rep, n_flat, u_feats_rep, neg_i_feats).view(B, K)
    neg_loss = F.binary_cross_entropy_with_logits(
        neg_logits, torch.zeros_like(neg_logits)
    )

    return pos_loss + neg_loss


In [ ]:
@torch.no_grad()
def evaluate_with_context(model, test_df, user_pos, n_items, user_feat_df, item_feat_df, k=10, n_neg=99):
    model.eval()
    HR, NDCG = [], []

    item_feat_mat = torch.tensor(item_feat_df[item_features].values, dtype=torch.float32, device=DEVICE)
    user_feat_mat = torch.tensor(user_feat_df[user_features].values, dtype=torch.float32, device=DEVICE)

    for u, pos in zip(test_df.u.values, test_df.i.values):
        u = int(u); pos = int(pos)
        s = user_pos[u]

        negs = []
        while len(negs) < n_neg:
            j = np.random.randint(0, n_items)
            if j != pos and j not in s:
                negs.append(j)

        items = np.array([pos] + negs, dtype=np.int64)
        users = np.full_like(items, u)

        uf = user_feat_mat[users]   # (1+n_neg, F_u)
        ifeat = item_feat_mat[items]# (1+n_neg, F_i)

        logits = model(
            torch.tensor(users, device=DEVICE),
            torch.tensor(items, device=DEVICE),
            uf, ifeat
        ).cpu().numpy()

        rank = logits.argsort()[::-1]
        topk = items[rank[:k]]

        HR.append(1 if pos in topk else 0)
        if pos in topk:
            idx = np.where(topk == pos)[0][0]
            NDCG.append(1 / np.log2(idx + 2))
        else:
            NDCG.append(0.0)

    return float(np.mean(HR)), float(np.mean(NDCG))


In [ ]:
# Loader
train_loader = DataLoader(
    ContextualImplicitDataset(train_df, n_items, user_pos, NEG_PER_POS),
    batch_size=BATCH_SIZE,
    shuffle=True,
    drop_last=False
)

# Model
model_adv = AdvancedNeuMF(
    n_users, n_items, len(user_features), len(item_features),
    eg=EMBED_GMF, em=EMBED_MLP, layers=MLP_LAYERS
).to(DEVICE)

opt = torch.optim.Adam(model_adv.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

# Item feature matrix for negatives (DEVICE)
item_feat_matrix = torch.tensor(
    item_feat_df[item_features].values, dtype=torch.float32, device=DEVICE
)

for e in range(1, EPOCHS + 1):
    model_adv.train()
    total = 0.0

    for u, p, n, uf, ifeat_pos in train_loader:
        u = u.to(DEVICE); p = p.to(DEVICE); n = n.to(DEVICE)
        uf = uf.to(DEVICE); ifeat_pos = ifeat_pos.to(DEVICE)

        opt.zero_grad()
        loss = context_bce(model_adv, u, p, n, uf, ifeat_pos, item_feat_matrix)
        loss.backward()
        opt.step()

        total += loss.item() * u.size(0)

    hr, ndcg = evaluate_with_context(
        model_adv, test_df, user_pos, n_items, user_feat_df, item_feat_df, k=TOPK
    )

    print(f"Epoch {e:02d} | train_bce={total/len(train_df):.4f} | HR@{TOPK}={hr:.3f} | NDCG@{TOPK}={ndcg:.3f}")


NameError: name 'item_feat_df' is not defined